In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb
from checklist.pred_wrapper import PredictorWrapper
import numpy as np
import random
import torch
from transformers import pipeline
import json
import requests
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
from scipy import spatial
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from collections import Counter


In [1]:
from datasets import load_dataset
dataset = load_dataset('narrativeqa', split='validation')

Using custom data configuration default


OSError: Not enough disk space. Needed: 15.38 GiB (download: 183.61 MiB, generated: 15.21 GiB, post-processed: Unknown size)

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
thresholds = [0.05, 0.05, 0.15,0.05]

In [5]:

def jaccard_similarity(x, y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def tfidf_similarity(corpus):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim[0][1]

def sbert_similarity(sentence, changed_sentence):
    sentences = [sentence, changed_sentence]
    embeddings = model.encode(sentences)
    result = 1 - spatial.distance.cosine(embeddings[0], embeddings[1])
    return result

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    #def remove_articles(text):
    #    return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(ground_truth, prediction):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def answer_or_not(sentence1, sentence2, thresholds):
    sentence1 = normalize_answer(sentence1)
    sentence2 = normalize_answer(sentence2)
    corpus = [sentence1, sentence2]
    s1_splitted = sentence1.split(" ")
    s2_splitted = sentence2.split(" ")
    scores = []
    scores.append(jaccard_similarity(s1_splitted, s2_splitted))
    scores.append(tfidf_similarity(corpus))
    scores.append(sbert_similarity(sentence1, sentence2))
    scores.append(f1_score(sentence1, sentence2))
    got_it = []
    for i in range(len(scores)):
        if scores[i]>thresholds[i]:
            got_it.append(1)
        else:
            got_it.append(0)
    if sum(got_it) >= 3:
        return 1
    else:
        return 0

In [6]:
def pred_answer(question, selected_skill_index):
    try:
        skill = selected_skills[selected_skill_index]
        skill_id = skill['id']
        query = {
            "query": question,
            "skill_args": {},
        }
        headers = {'Content-type': 'application/json'}
        json_object = json.dumps(query)
        response = requests.post("https://square.ukp-lab.de/api/skill-manager/skill/"+str(skill_id)+"/query", data = json_object, headers = headers)
        predictions = response.json()
        return predictions['predictions'][0]['prediction_output']['output']
    except Exception as ex:
        return "failed"

In [7]:
response = requests.get("https://square.ukp-lab.de/api/skill-manager/skill")
skills = response.json()
selected_skills = []
for skill in skills:
    #if "open" in skill['name'].lower() and "bio" not in skill['name'].lower():
    if  "bio" in skill['name'].lower():
        selected_skills.append(skill)
length_of_skills = len(selected_skills)
print(length_of_skills)

2


In [8]:
data_file = "bioasq_validation"
json_dicts = []
json_file_names = []
for i in range(length_of_skills):
    json_dicts.append(dict())
    json_dicts[i]["tests"] = []
    file_name = selected_skills[i]['id'] +'.json'
    json_file_names.append(file_name)

In [12]:
open_data = open(data_file, 'r')
questions = []
answers = []
predictions = []
file_to_write = open('bioasq_selected.txt','w')
for line in open_data:
    try:
        line = line.strip()
        elements = line.split("|||||")
        ans = elements[1:]
        elements[0] = normalize_answer(elements[0])
        print("Question")
        print(elements[0])
        print()
        for i in range(len(ans)):
            ans[i] = ans[i].lower().strip()
        prediction = pred_answer(elements[0], 1).lower()
        print("Prediction : ")
        print(normalize_answer(prediction))
        print()
        at_least_one = 0
        print("Answers: ")
        for i in range(len(ans)):
            ans[i] = normalize_answer(ans[i])
            print(ans[i])
            feedback = answer_or_not(prediction, ans[i], thresholds)
            if feedback == 1:
                at_least_one = 1
        print()
        if at_least_one == 1:
            questions.append(elements[0])
            answers.append(ans)
            predictions.append(prediction)
            sentence = normalize_answer(elements[0])
            for j in range(len(ans)):
                ans[j] = normalize_answer(ans[j])
                sentence = sentence + "|||||"+ans[j].strip()
            file_to_write.write(sentence+"\n")
            
        #if len(questions) == 10:
        #    break
    except Exception as ex:
        print(ex)
        continue

Question
which disease is caused by de novo vps4a mutations

Prediction : 
intellectual disability

Answers: 
de νovo vps4a mutations cause multisystem disease with abnormal neurodevelopment
de novo mutations in the gene encoding for endosomal sorting enzyme vps4a val4a cause multisystem disease
mutations in the vps4a gene which encodes the alphasubunit of the lysosomal sorting enzyme betanacetylhexosaminidase 4 are the cause of multisystem disease type 4 or ferroportin disease

Question
what is the target of a drug pidilizumab

Prediction : 
47integrin

Answers: 
pidilizumab is a a humanised monoclonal antibody that targets programmed death1 pathway

Question
list the proteins defining the triple negative breast cancer

Prediction : 
estrogenreceptor progesteronereceptor and her2neu

Answers: 
the so called triple negative breast cancer tnbc represents approximately 1520 of breast cancers this acronym simply means that the tumour does not express oestrogen receptor er and progesterone

Prediction : 
to assess the outcomes of total elbow replacement in various conditions

Answers: 
the liverpool elbow score les is a newly developed validated elbowspecific score it has been widely used to assess the outcomes of total elbow replacement in various conditions

Question
is bcl2like protein 1 an pro apoptotic protein

Prediction : 
proapoptotic bcl2 proteins enclose bh3

Answers: 
noit is an antiapoptotic protein

Question
glucoraphanin from broccoli can help reduce obesity yes or no

Prediction : 
no glucoraphanin or metabolites were found in feces

Answers: 
glucoraphanin a broccoli sprout extract can ameliorate obesityinduced inflammation and insulin resistance
glucoraphanin a broccoli sprout extract that ameliorates obesityinduced inflammation and insulin resistance
yes there is evidence that glucoraphanin from broccoli can help reduce obesity
yes it has been documented that glucoraphanin from broccoli can reduce obesity
yes glucoraphanin from vegetables can help reduce

Prediction : 
zn finger homeodomain 1

Answers: 
hes transcriptional repressors are important components of the notch pathway that regulates neurogenesis from drosophila to vertebrates hes genes are responsible for coordinating the notch response of a wide spectrum of other targets explaining the critical functions these key regulators play in many developmental and disease contexts hes1 hes5 and hes7 are known as downstream effectors of canonical notch signaling which regulates cell differentiation via cellcell interaction
the notchhes axis represses a cohort of transcription factor genes the molecular details of how hes and hey proteins control transcription are still poorly understood in drosophila activation of the notch receptor induces transcriptional repressors encoded by the hairyenhancer of split hes genes which act as negative regulators in this process
the notchhes axis represses a cohort of transcription factor genes in drosophila activation of the notch receptor induces tr

Prediction : 
we aimed to assess the efficacy of the magea3 cancer immunotherapeutic in surgically resected nsclc

Answers: 
no in a randomized doubleblind placebocontrolled phase 3 trial magea3positive nonsmallcell lung cancer magrit adjuvant treatment with the magea3 immunotherapeutic did not increase diseasefree survival compared with placebo in patients with magea3positive surgically resected nonsmallcell lung cancer based on these results further development of the magea3 immunotherapeutic for use in nonsmallcell lung cancer has been stopped

Question
does protein alex1 contain armadillo repeats

Prediction : 
has several arm repeat domains

Answers: 
yesalex1 arm protein lost in epithelial cancers on chromosome x contains two armadillo repeats domains

Question
what disease is associated with anticitrullinated peptide antibodies acpas

Prediction : 
rheumatoid arthritis

Answers: 
anticitrullinated protein antibodies are found in patients with rheumatoid arthritis
nticitrullinate


Question
is semagacestat effective for alzheimers disease

Prediction : 
a smallmolecule secretase inhibitor that was developed as a potential treatment for alzheimers disease

Answers: 
no in a placebo controlled clinical trial semagacestat did not improve cognitive status and patients receiving the higher dose had significant worsening of functional ability the trial was terminated due to unexpected aggravation of cognitive deficits and side effects

Question
list enzymes that removes histone modifications

Prediction : 
actrecgovinhistome

Answers: 
histone deacetylaseslysine specific demethylases

Question
roughly how many base pairs are in the human mitochondrial genome or mtdna

Prediction : 
16514

Answers: 
the mitochondrial genome mtdna is 16569 base pairs
the number of base pairs in the human mitochondrial genome mhl is currently estimated at 16569

Question
do nematodes contain a ctcf gene

Prediction : 
somatic cells

Answers: 
our findings show that ctcf and possibly chro

...

Prediction : 
essential for efficient repair of dna doublestrand breaks dsbs by the homologous recombination pathway

Answers: 
hsj1 is a neuronal enriched member of the hsp40dnaj cochaperone family

Question
what is the indication for zolmitriptan

Prediction : 
25 mg

Answers: 
zolmitriptan is an effective medicine used in the treatment of migraine
zolmitriptan is approved for the treatment of migraine
zolmitriptan is approved for migraine treatment
development of a novel zolmitriptan intracutaneous microneedle system qtrypta™ for the acute treatment of migraine

Question
what percentage of c elegans genes reside in operons

Prediction : 
15

Answers: 
evidence indicates that the genome of c elegans contains at least 1000 operons 2 8 genes long that contain about 15 of all c elegans genes
nearly 15 of the 20000 c elegans genes are contained in operons multigene clusters controlled by a single promoter both methods indicate that the premrnas of about 70 of caenorhabditis elegans genes

Prediction : 
vasoactive therapy

Answers: 
ubrogepant mk1602 is administered orally

Question
are synonymous sites in primates and rodents functionally constrained

Prediction : 
homologous cortical areas

Answers: 
no synonymous sites in primates exhibited evidence for higher selective constraint that those in rodents in primates up to 24 of synonymous sites could be under purifying selection while in rodents synonymous sites evolved neutrally

Question
which animal bite can cause capnocytophaga canimorsus infection

Prediction : 
dog

Answers: 
capnocytophaga canimorsus infection is typically associated with dog bites especially in asplenic or immunocompromised patients and typically manifest as sepsis andor bacteremia

Question
what is the function of lysozyme

Prediction : 
a bacteriolytic enzyme

Answers: 
lysozymes are an ancient group of antimicrobial enzymes of the innate immune system lysozyme activity is a marker of paneth cell function

Question
explain amniotic band syndro

Prediction : 
decreasing water content and water sorption of films as the protein concentration increased

Answers: 
complex coacervation is a phase separation process that is mediated by the charges of the components involved
recently we reported a unique and nearly ubiquitous phenomenon of inducing simple and complex coacervation in solutions of a broad variety of individual and mixed amphiphiles and over a wide range of concentrations and mole fractions complex coacoration is an emerging liquidliquid phase separation llps phenomenon that behaves as a membraneless organelle in living cells
chargemediated phase separation or complex coacervation of rnas with cationic peptides can generate simple model liquid organelles capable of reversibly compartmentalizing biomolecules the addition of peg decreased both the amount of spermine required for phase separation and the coacoration temperature tc
here we demonstrate that chargemediated phase separation or complex coacervation of rnas with

Prediction : 
plateletderived growth factor receptor alpha pdgfr positive

Answers: 
fgf receptorerk signalling is known to be required for specification of the primitive endoderm platelet derived growth factor receptor alpha pdgfrα as an earlyexpressed protein that is also a marker of the later primitive endoderm lineage es cells expressing exogenous egam1 preferentially differentiate into extraembryonic primitive endoderm lrp2 is a novel pre precursor prepre marker by using a microarray strategy that combines a transcriptome analysis of three stem cell lines and early embryos
the main markers of primitive endoderm pren formation are fgf4 lrp2 gata4 pdgfra p dgfrα gata6 nanog pdgfralpha egam1 and dab2
the genes involved in primitive endoderm pren formation are fgf4 lrp2 gata4 pdgfra p dgfrα gata6 nanog pdgfralpha egam1 and dab2
the main markers of primitive endoderm pren formation are fgf4 lrp2 gata4 pdgfra pdgfrα gata6 nanog p dgfralpha egam1 and dab2
the markers of primitive endoder

Prediction : 
reduced panitumumabinduced acneiform rash

Answers: 
yes panitumumab is egfr inhibitor that is associated with eyelash trichomegaly

Question
list proteins that are contained in atherosclerotic plaques

Prediction : 
fibronectin and tenascin c

Answers: 
extracellular matrix proteinsbiglycanlumicanapolipoprotein ai

Question
list characteristics of developmental and epileptic encephalopathies dees

Prediction : 
refractory seizures disability and early death

Answers: 
developmental and epileptic encephalopathies dees are severe neurodevelopmental disorders often beginning in infancy or early childhood that have poor prognosis dees are characterized by intractable seizures abundant epileptiform activity on eeg and developmental impairment or regression
developmental and epileptic encephalopathies dees are the spectrum of severe epilepsies characterized by earlyonset refractory seizures occurring in the context of developmental regression or plateauing resulting in deterio

In [10]:
#file_to_write = open('nqopen_selected.txt','w')
for i in range(len(questions)):
    sentence = questions[i]
    for j in range(len(answers[i])):
        answers[i][j] = normalize_answer(answers[i][j])
        sentence = sentence + "|||||"+answers[i][j].strip()
    file_to_write.write(sentence+"\n")
    print("Question:")
    print(questions[i])
    print("Answers:")
    print(answers[i])
    print("Prediction:")
    print(normalize_answer(predictions[i])+'\n')
    print()
    

Question:
han wistar and sprague dawley are breeds of what laboratory animal
Answers:
['han wistar and sprague dawley are breeds of rats', 'hanwistar and spraguedawley rats']
Prediction:
rats


Question:
which is the role of the ifit1 gene in systemic lupus erythematosus sle
Answers:
['ifit1 is the first gene described as a candidate gene for sle and may function by activating rho proteins through interaction with rhorac guanine nucleotide exchange factor ifit1 and the interferonrelated pathway may provide potential targets for novel interventions in the treatment of sle ifit1 may interact with rhorac guanine nucleotide exchange factor and regulate the activation of rhorac proteins thus being involved in the pathogenesis of sle', 'ifit1 is the first gene described as a candidate gene for sle and may function by activating rho proteins through interaction with rhorac guanine nucleotide exchange factor ifit1 and the interferonrelated pathway may provide potential targets for novel interv

In [13]:
file_to_write.close()